There is a great deal of silicate vapor (also referred to as gas) in the moon-forming region of a synestia. Any growing silicate moonlets (also referred to as condensates or liquid droplets) will orbit within the moon-forming region of a synestia and will encounter this gas. The gas will impart a drag upon the moonlets, causing them to slow down and lose angular velocity. The orbit of the moonlets will decay over time in accord with the strength of the gas drag acceleration.

In [ ]:
#INSESRT DIAGRAM HERE

Let's get a sense for the magnitude of gas drag acceleration that we're dealing with. It is assumed that the moonlets experience a simple gas drag. The equation is as follows:
$$a_{drag}=\frac{C_D\ A \ (u_{gas} - u_{body})^2}{2 \ m_{body}}$$

Simple gas drag acceleration a$_{drag}$ is dependent on: the mass of the body that is being dragged by the gas $m_{body}$, the cross sectional area of the body that the gas is interacting with A, the gas drag coefficient C$_D$ (what thermal regime we are in), and the relative velocity between the gas and the body being dragged u$_{gas}$-u$_{body}$. Later, we calculate the gas drag coefficient using the Adachi et al (1976) formulation, but here you can play with its value.

In [113]:
import numpy as np
import math
from ipywidgets import *
import matplotlib.pyplot as plt
v_rel = np.arange(0.0, 300., 1.) #relative velocity between gas and body in m/s
Moon_density = 3300. #lunar density in kg/m3; 3.3 g/cc
def pltdrag(rad_body, C_D):
    mass_body = Moon_density*(4./3.)*math.pi*(rad_body**3) #mass of body in kg; assume spherical body
    a_drag = 0.5*C_D*math.pi*(rad_body**2)*(v_rel**2)/mass_body #acceleration imparted on body due to gas drag in m/s2; assume spherical cross section
    plt.plot(v_rel, a_drag)
    plt.xlabel('velocity of the gas relative to the velocity of the moonlet (m/s)')
    plt.ylabel('gas drag acceleration (m/s$^2$)')
    plt.show()

#PLOT ACCELERATION RELATIVE TO SOMETHING ELSE (g's, roller coasters, astronauts, etc.)    
#RELATIVE SCALE IN GENERAL: how big the moonlets are (marble, golf ball, grain of sand, yoga ball, soccer field, etc.), how fast the relative and absolute velocity is
#MAYBE THIS SHOULD BE PRINTED OUTPUT?    
style = {'description_width': 'initial'}
layout = {'width': '400px'}
%matplotlib inline
interact(pltdrag, rad_body=FloatLogSlider(value=1, base=10, min=-3, max=6, step=1, description='Size of moonlet (m)', style=style, layout=layout),
         C_D=FloatSlider(value=2, min=0, max=2.5, step=0.5, description='Gas drag coefficient C$_D$ (-)', continuous_update=False, readout=True, readout_format='.1f', style=style, layout=layout))

interactive(children=(FloatLogSlider(value=1.0, description='Size of moonlet (m)', layout=Layout(width='400px'…

<function __main__.pltdrag(rad_body, C_D)>

As expected, gas drag acceleration is greater when the mass of the moonlet is smaller and the gas drag coefficient and relative velocity are larger.

Now, let's see how a single moonlet travels through the vapor in a synestia. First, we will look at how a moonlet moves on a stable Keplerian orbit. Next, we will see the effects of the second-order term of gravity on the orbit of the moonlet. Moonlets, once formed, will quickly equilibrate to the midplane. So, we will look at orbits in the midplane first. Moonlets condense as droplets at the photosphere (where the vapor is saturated and has a pressure that lies on the vapor dome). We will check orbits of moonlets condensing at the photosphere (starting off the midplane) next. Let's check the decay time for various sizes of moonlets and various starting semi-major axes.

In [116]:
#CALCULATE KEPLERIAN ORBIT WITH NORMAL GRAVITY
#G=6.674e-11 #SI
%run synfits.py
Mass_syn=np.sum(SNAP_Canup.m) #kg #Earth mass
from mpl_toolkits.mplot3d import Axes3D
def pltKep(x,y,z,n):
    #n counter-clockwise orbits
    #default starts at (x,y,z)
    #with initial velocity (v_x,v_y,v_z)
    t=0. #initial time in seconds
    radius = (x**2 + y**2 + z**2)**0.5 #distance from origin of central body (m)
    P=((radius**3)*4.*math.pi*math.pi/(G*Mass_syn))**0.5 #orbital period (s)
    dt=P/100. #time step (s)
    time=P*n #cumulative time of orbit (s)
    vel=np.sqrt(G*Mass_syn/radius) #total velocity (m/s)
    vx=-vel*y/radius #x vel. (m/s)
    vy=vel*x/radius #y vel. (m/s)
    vz=vel*z/radius #z vel. (m/s)
    xarr=[]
    yarr=[]
    zarr=[]
    tarr=[]
    xarr.append(x)
    yarr.append(y)
    zarr.append(z)
    tarr.append(t)
    while (t < time):
        radius = (x**2 + y**2 + z**2)**0.5 #distance from origin of central body (m)
        accel_grav=-G*Mass_syn/(radius**3) #total acceleration (m/s^2)
        ax=accel_grav*x #x accel. (m/s^2)
        ay=accel_grav*y #y accel. (m/s^2)
        az=accel_grav*z #z accel. (m/s^2)
        vx+=ax*0.5*dt
        vy+=ay*0.5*dt
        vz+=az*0.5*dt
        radius = (x**2 + y**2 + z**2)**0.5 #distance from origin of central body (m)
        accel_grav=-G*Mass_syn/(radius**3) #total acceleration (m/s^2)
        ax=accel_grav*x #x accel. (m/s^2)
        ay=accel_grav*y #y accel. (m/s^2)
        az=accel_grav*z #z accel. (m/s^2)
        vx+=ax*0.5*dt
        vy+=ay*0.5*dt
        vz+=az*0.5*dt
        x+=vx*dt
        y+=vy*dt
        z+=vz*dt
        t+=dt
        xarr.append(x)
        yarr.append(y)
        zarr.append(z)
        tarr.append(t)
    xarr=np.asarray(xarr)
    yarr=np.asarray(yarr)
    zarr=np.asarray(zarr)
    radiusarr=np.sqrt(xarr**2 + yarr**2 + zarr**2)
    tarr=np.asarray(tarr)
    fig=plt.figure(figsize=(9,3))
    plot1 = fig.add_subplot(121, projection='3d')
    plot1.plot(xarr,yarr,zarr,color='b')
    plot1.set_xlabel('x (m)')
    plot1.set_ylabel('y (m)')
    plot1.set_zlabel('z (m)')
    plot2 = fig.add_subplot(122)
    plot2.plot(tarr/3600,radiusarr,color='b')
    plot2.set_xlabel('time (hr)')
    plot2.set_ylabel('radius (m)')
    plt.show()
#%matplotlib notebook    
interact(pltKep,
         x=FloatSlider(value=20e6, min=10e6, max=60e6, step=10e6, description='Starting x distance from central body (m)',
                        continuous_update=False, readout=True, readout_format='.1f', style=style, layout=layout),
         y=FloatSlider(value=0, max=60e6, step=10e6, description='Starting y distance from central body (m)',
                        continuous_update=False, readout=True, readout_format='.1f', style=style, layout=layout),
         z=FloatSlider(value=0, max=60e6, step=10e6, description='Starting z distance from the midplane (m)',
                        continuous_update=False, readout=True, readout_format='.1f', style=style, layout=layout),
         n=IntSlider(value=1, max=10, step=1, description='Number of orbits',
                        continuous_update=False, readout=True, readout_format='d', style=style, layout=layout))  

Read 180233 particles from TE_Example01_Cool05_snapshot_4096_long
Read 201779 particles from TE_Example03_Cool01_snapshot_10500_long
Read 100989 particles from TE_Example07_CoolB01_snapshot_7200_long


<Figure size 432x288 with 0 Axes>

interactive(children=(FloatSlider(value=20000000.0, continuous_update=False, description='Starting x distance …

<function __main__.pltKep(x, y, z, n)>

In [ ]:
#IMPORT GADGET-2 SNAPSHOT (VAPOR IN SYNESTIA)
#FIT THERMAL PARAMETER PROFILES: PRESSURE, TEMPERATURE, DENSITY
#FIT IN-PLANE VELOCITY PROFILE
#CALCULATE ORBITAL DECAY; UPDATE ACCELERATION DUE TO DRAG
widgets.Dropdown(
    options=[('Thickness of a plastic ID card (mm)', 1.e-3),
             ('Diameter of a belly button (cm)', 1.e-2),
             ('Coconut (dm)', 0.1),
             ('Cow (m)', 1.),
             ('230 story building (km)', 1.e3),
             ('Width of the United States (Mm)', 1.e6),
             ('The Moon (3.5 Mm)', 3.5e6)],
    value=1,
    description='Size (diameter) of moonlet:',
)
#need leapfrog integrator

ModuleNotFoundError: No module named 'rebound'

In [ ]:
#REFERENCES